## About This Notebook

This is a playground notebook of sorts where I can try to implement a lot of the code from the assignments from scratch.

The assignments are helpful but there are too many hints and it's too easy for someone to work through them without truely understanding how transformers, attention, etc. works.

In [1]:
import sys
import os

import numpy as np

import textwrap
wrapper = textwrap.TextWrapper(width=70)

import trax
from trax import layers as tl
from trax.fastmath import numpy as jnp

# to print the entire np array
np.set_printoptions(threshold=sys.maxsize)

### Scale Dot Product Attention

In [ ]:
def scaled_dp_attention(q, k, v, m, embed_dims, vocab_size):
    '''
    k - keys [batch, seq_len_in, embed_dims]
    q - query [batch, seq_len_out, embed_dims]
    v - values [batch, seq_len_in, embed_dims]
    m - mask [batch, seq_len_out, seq_len_in]
    '''
    kt = jnp.transpose(k)
    w = tl.Softmax(jnp.dot(q, kt)/sqrt(embed_dims))
    return jnp.dot(w,v)
    
    #LEFT OFF HERE - this is wrong but not horribly so.  Softmax is not correct. No mask.
    
    #Note: it's a little unclear to me how I can use trax layers with jax.numpy operations. 